In [105]:
import re

import joblib
import pandas as pd
from sklearn.impute import SimpleImputer

mails_df = pd.read_csv("../data/training_data.csv", sep='~', index_col=0)
imputer = SimpleImputer(strategy='constant', fill_value='')
mails_df = pd.DataFrame(imputer.fit_transform(mails_df), columns=mails_df.columns)
# mails_df = mails_df.iloc[:10]

In [106]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
import re

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

In [107]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub('[^a-zA-Z]', ' ', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    processed_text = ' '.join([lemmatizer.lemmatize(token) for token in tokens])
    return processed_text


def preprocess_sender(address):
    address = address.lower()
    address = re.sub('[.]ac[.]in|[.]com', '', address)
    address = re.sub('@|[.]', ' ', address)
    return address

preprocessor = ColumnTransformer(transformers=[
    ('subject', TfidfVectorizer(preprocessor=preprocess_text, min_df=0.01), 'subject'),
    ('body', TfidfVectorizer(preprocessor=preprocess_text, max_df=0.9, min_df=0.1), 'body'),
    ('sender', TfidfVectorizer(preprocessor=preprocess_sender), 'sender')
], remainder='drop')

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [108]:
preprocess_sender('hello@cs.iitr.ac.in')

'hello cs iitr'

In [109]:
prepeprocessed_df = pipeline.fit_transform(mails_df.iloc[:8])
prepeprocessed_df

<8x594 sparse matrix of type '<class 'numpy.float64'>'
	with 737 stored elements in Compressed Sparse Row format>

In [110]:
pipeline.get_feature_names_out()

array(['subject__academic', 'subject__award', 'subject__call',
       'subject__championship', 'subject__conversation',
       'subject__convocation', 'subject__facility', 'subject__final',
       'subject__fraya', 'subject__frehse', 'subject__fwd',
       'subject__gaurav', 'subject__gradesheet', 'subject__institute',
       'subject__intro', 'subject__lake', 'subject__macay',
       'subject__notice', 'subject__open', 'subject__opportunity',
       'subject__ph', 'subject__prof', 'subject__quiz', 'subject__raheja',
       'subject__request', 'subject__room', 'subject__scholarship',
       'subject__semester', 'subject__spic', 'subject__student',
       'subject__study', 'subject__talk', 'subject__tehri',
       'subject__trip', 'subject__ug', 'subject__usa', 'subject__video',
       'subject__washing', 'body__aakash', 'body__aao', 'body__aastha',
       'body__abdul', 'body__ac', 'body__academic', 'body__accepting',
       'body__achievement', 'body__across', 'body__action',
       '

In [111]:
pp1 = pipeline.transform(mails_df.iloc[8:])
pp1

<1118x594 sparse matrix of type '<class 'numpy.float64'>'
	with 31571 stored elements in Compressed Sparse Row format>

In [112]:
from sklearn.preprocessing import MultiLabelBinarizer
import json

with open('../data/label_dict.json', 'r') as file:
    all_labels = json.load(file)
label_list = [key for key in all_labels.keys() if re.match('Label_[0-9]', key)]
mlb = MultiLabelBinarizer(classes=label_list)
labels_array = [list(st.split(',')) for st in mails_df['labels']]
mlb.fit(label_list)
labels = mlb.transform(labels_array)

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['CATEGORY_FORUMS', 'CATEGORY_PERSONAL', 'CATEGORY_PROMOTIONS', 'CATEGORY_UPDATES', 'IMPORTANT', 'INBOX', 'SENT', 'STARRED', 'UNREAD'] will be ignored
  warnings.warn(


In [113]:
feature_matrix = pipeline.fit_transform(mails_df)
feature_matrix

<1126x399 sparse matrix of type '<class 'numpy.float64'>'
	with 27062 stored elements in Compressed Sparse Row format>

In [114]:
X = feature_matrix
y = labels
print(X.shape, y.shape)

(1126, 399) (1126, 21)


In [137]:
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans

k = 25
model = KMeans(n_clusters=k, random_state=42)
labels = model.fit_predict(X.toarray())
labels

C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\msing\.conda\envs\gmail_organizer\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=5.
  warnings.warn(


array([16, 22,  1, ...,  1, 21, 18])

In [140]:
mails_df['label'] = pd.Series(labels, name='label')

In [141]:
mails_df.loc[:,['subject', 'label']]

,subject,label
0,Video of Prof. Gaurav Raheja in conversation w...,16
1,Fwd: Institute Open Championship,22
2,Study and Scholarship opportunities in USA,1
3,Re: Request for semester 1 UG academic gradesheet,17
4,SPIC MACAY Intro Talk & Quiz,19
...,...,...
1121,Prof R J Garde Endowment Lecture,23
1122,Updates: Your List of Quarantined Emails since...,6
1123,You have been added to a team in Microsoft Teams,1
1124,Re: Regarding the updated mobile no. and room no.,21
